In [22]:
from zhipuai import ZhipuAI
import os

os.environ['ZHIPUAI_API_KEY'] = '59254518d3e99d10be3772c92a8b06bb.3k0zKQtAy6I8C1r2'

client = ZhipuAI()


def get_completions(prompt):
    messages=[
        {
            'role': 'user',
            'content': prompt,
        }
    ]
    response = client.chat.completions.create(
        model='glm-4',
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

instruction = '''你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的倾向。'''

input = '办个校园卡，要求100G套餐,价格尽量低'

prompt = f'{instruction} 用户输入:{input}'

print(get_completions(prompt))


根据用户的输入，我们可以分析出以下倾向：

1. **名称**：用户倾向于选择“校园卡”类型的流量套餐。这可能意味着用户希望选择专为在校学生或与学校有关的人群设计的套餐。

2. **月流量**：用户明确要求了“100G”的月流量。这表明用户对流量需求较高，希望获得一个大流量的套餐。

3. **月费价格**：用户表示“价格尽量低”，这表明用户在满足流量需求的前提下，对价格非常敏感，希望得到性价比高的流量套餐。

综上所述，用户在选择流量套餐产品时，最看重的是月流量的大小，其次是价格，希望以最低的成本获得100G的月流量，而对套餐的名称（即面向的用户群体）也有明确的要求，希望是针对校园用户的套餐。


In [21]:
output_format='用json格式输出'

prompt = f'{instruction} {output_format}  用户输入:{input} '
print(get_completions(prompt))

根据用户输入，我们可以分析出以下倾向：

- 用户倾向于选择“校园卡”类型的流量套餐。
- 用户希望月流量至少为100G。
- 用户对月费价格的要求是“尽量低”。

以下是按照用户要求以JSON格式输出的示例：

```json
{
  "倾向": {
    "套餐类型": "校园卡",
    "月流量": "100G",
    "月费价格": "最低"
  }
}
```

或者，如果需要更详细地表示每个属性的具体值（尽管用户没有提供确切的价格），可以如下表示：

```json
{
  "用户选择": {
    "名称": "校园卡",
    "月费价格": {
      "要求": "最低"
    },
    "月流量": "100G"
  }
}
```

请注意，由于用户没有提供确切的价格范围，我们只能表示用户希望找到最低价格的套餐。实际的价格需要根据市场情况来确定。


## 把输出格式定义的更精细

In [35]:
instruction='''
以JSON格式输出。
1. name字段的取值为string类型，取值必须为以下之一：经济套餐、畅游套餐、无限套餐、校园套餐 或 null；

2. price字段的取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型

3. data字段的取值为取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型或string类型，string类型只能是'无上限'

4. 用户的意图可以包含按price或data排序，以sort字段标识，取值为一个结构体：
(1) 结构体中以"ordering"="descend"表示按降序排序，以"value"字段存储待排序的字段
(2) 结构体中以"ordering"="ascend"表示按升序排序，以"value"字段存储待排序的字段

只输出中只包含用户提及的字段，不要猜测任何用户未直接提及的字段，不输出值为null的字段。
'''



prompt = f"""
{instruction}

{output_format}

用户输入：
{input}
"""
input='我要无限量套餐'

response = get_completions(prompt)
print(response)

基于用户输入，我们可以构建以下JSON格式的响应：

```json
{
  "name": null,
  "price": {
    "operator": "<=",
    "value": null
  },
  "sort": {
    "ordering": "ascend",
    "value": "price"
  }
}
```

在这个例子中，因为用户没有指定套餐类型（name字段），所以该字段为null。用户询问“有没有便宜的套餐”，表明他们对价格有要求，但并没有给出具体的价格值，因此price结构体中的value字段也为null。同时，用户意图是按照价格排序，且是按照升序（便宜到贵）排序。

对于第二个场景：

```json
{
  "category": null,
  "effect": null,
  "price": {
    "operator": "<=",
    "value": null
  },
  "sort": {
    "ordering": "ascend",
    "value": "price"
  }
}
```

同样的逻辑适用于这个场景，用户没有指定具体的类别（category字段）和效果（effect字段），只对价格有要求，因此这两个字段为null。价格条件与上面的场景相同，并且用户希望按照价格升序排序。

请注意，如果用户提供了具体的价格值或者有其他具体要求，JSON中的相应字段应当包含这些信息。
